# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import os
import joblib
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
from pprint import pprint # will use in printing automl model parameters
from azureml.core import Model # will use to get model information

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'CapstoneProject'

experiment=Experiment(ws, experiment_name)

In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data = 'https://raw.githubusercontent.com/Basit040/Capstone-Azure-Machine-Learning/main/titanicdataset.csv'
ds = TabularDatasetFactory.from_delimited_files(data)


In [ ]:
from train import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
# Data will be cleaned from clean function as writte in train.py script
x, y = clean_data(ds)

# Create a dataframe from the cleaned data
df = pd.concat([x, y], axis = 1)
df.info()

In [10]:
ds.head()

AttributeError: 'TabularDataset' object has no attribute 'head'

In [ ]:
df=df.to_pandas_dataframe()

In [ ]:
from sklearn.model_selection import train_test_split


#Split data into training and test and create a csv files with the result 
df_train, df_test = train_test_split(df, test_size=0.35)
df_train.to_csv("training/training_dataset.csv")
df_test.to_csv("validation/validation_dataset.csv")

In [ ]:
#Create an experiment in the workspace
experiment = Experiment(ws, "auto_ml_titanic")

#Get the default datastore for the workspace.
datastore = ws.get_default_datastore()

In [ ]:
#Upload the training dataset and the validation dataset to the datastore 
datastore.upload(src_dir = "training/", target_path = "data/")
datastore.upload(src_dir = "validation/", target_path = "data/")

In [ ]:
# Upload the training data and the validation data as a tabular dataset 
training_data = TabularDatasetFactory.from_delimited_files(path = [(datastore, ("data/training_dataset.csv"))])
validation_data = TabularDatasetFactory.from_delimited_files(path = [(datastore, ("data/validation_dataset.csv"))])

In [5]:
# Compute name should contain only letters, digits, hyphen and should be 2-16 charachters long
compute_name = "BasCompute"
try:
    trainCluster = ComputeTarget(ws, compute_name)
    print(f"{compute_name} exists already")
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=5)
    trainCluster = ComputeTarget.create(ws, compute_name, compute_config)
trainCluster.wait_for_completion(show_output=True)

BasCompute exists already

Running


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [11]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 25,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here, since i will be deploying a ONNX model, enable_onnx is set to true
automl_config = AutoMLConfig(
    task='classification',
    compute_target=trainCluster,
    training_data=ds,
    label_column_name='Survived',
    n_cross_validations=4,
    **automl_settings)

In [12]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [13]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardi

{'runId': 'AutoML_fe744270-a7fe-4c59-b29f-79cd3cf0a17a',
 'target': 'BasCompute',
 'status': 'Completed',
 'startTimeUtc': '2021-02-05T15:09:04.600927Z',
 'endTimeUtc': '2021-02-05T15:41:41.687972Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '4',
  'target': 'BasCompute',
  'AMLSettingsJsonString': '{"path":null,"name":"CapstoneProject","subscription_id":"1b944a9b-fdae-4f97-aeb1-b7eea0beac53","resource_group":"aml-quickstarts-137332","workspace_name":"quick-starts-ws-137332","region":"southcentralus","compute_target":"BasCompute","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":4,"y_min":null,"y_max":null,"num_classes":null,"fe

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [14]:
# Get best run and model
best_run, fitted_model = remote_run.get_output()

# Print the best run
print(best_run)

# Get all metrics of the best run
best_run_metrics = best_run.get_metrics()

# Print all metrics of the best run
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Run(Experiment: CapstoneProject,
Id: AutoML_fe744270-a7fe-4c59-b29f-79cd3cf0a17a_94,
Type: azureml.scriptrun,
Status: Completed)
recall_score_macro 0.8318194746995853
average_precision_score_macro 0.8677293906812715
matthews_correlation 0.6728654575959481
AUC_macro 0.8689610356762488
precision_score_micro 0.8473871853916697
AUC_micro 0.8887762202547596
f1_score_micro 0.8473871853916697
AUC_weighted 0.8689610356762489
average_precision_score_weighted 0.8718862342275417
recall_score_weighted 0.8473871853916697
precision_score_weighted 0.8480286932782016
average_precision_score_micro 0.8761013680834401
weighted_accuracy 0.8610327573161375
f1_score_macro 0.8350349429653561
norm_macro_recall 0.6636389493991706
precision_score_macro 0.8412058275456401
f1_score_weighted 0.8463802348140474
recall_score_micro 0.8473871853916697
log_loss 0.41457089988670015
balanced_accuracy 0.8318194746995853
accuracy 0.8473871853916697
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_fe744270-a7fe-4c5

In [15]:
#TODO: Save the best model
# Print detailed parameters of the fitted model
def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(fitted_model)

datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}

prefittedsoftvotingclassifier
{'estimators': ['85',
                '14',
                '87',
                '44',
                '37',
                '92',
                '7',
                '27',
                '48',
                '71',
                '40'],
 'weights': [0.15384615384615385,
             0.07692307692307693,
             0.07692307692307693,
             0.07692307692307693,
             0.07692307692307693,
             0.07692307692307693,
             0.07692307692307693,
             0.07692307692307693,
             0.15384615384615385,
             0.07692307692307693,
             0.07692307692307693]}

85 - maxabsscaler
{'cop

In [16]:
#TODO: Save the best model
myModel = best_run.register_model(model_path='outputs/model.pkl', model_name='capstoneModel_automl',
                        tags={'Training context':'Auto ML'},
                        properties={'Accuracy': best_run_metrics['accuracy']})

print(myModel)

Model(workspace=Workspace.create(name='quick-starts-ws-137332', subscription_id='1b944a9b-fdae-4f97-aeb1-b7eea0beac53', resource_group='aml-quickstarts-137332'), name=capstoneModel_automl, id=capstoneModel_automl:1, version=1, tags={'Training context': 'Auto ML'}, properties={'Accuracy': '0.8473871853916697'})


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [17]:
# Download scoring file 
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'scoreScript.py')

# Download environment file
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'envFile.yml')

In [18]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script='scoreScript.py',
                                    environment=best_run.get_environment())

# deploy
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "myservice", [myModel], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

print(service.scoring_uri)

print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..........................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://0bc67f63-d44c-4ffd-9d95-b61755034a6b.southcentralus.azurecontainer.io/score
http://0bc67f63-d44c-4ffd-9d95-b61755034a6b.southcentralus.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service